In [45]:
library(tidyverse)
library(tidymodels)
library(dplyr)
library(cowplot)
library(ggplot2)
library(broom)

In [53]:
data <- read.csv("listings.csv", header = TRUE) %>%
    select(-id, -name, -host_id, -host_name, -host_since, -neighbourhood_cleansed, -property_type, -license)
data <- na.omit(data)
data["beds_per_accomodate"] = data$accommodates / data$beds
data <- data %>%
    select(-beds, -accommodates)
data <- data %>%
    filter(host_response_time != 'N/A')
head(data)

,host_response_time,host_response_rate,host_is_superhost,neighbourhood_group_cleansed,latitude,longitude,room_type,bathrooms,bedrooms,price,minimum_nights,availability_365,number_of_reviews,review_scores_rating,instant_bookable,beds_per_accomodate
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<dbl>
1,a few days or more,0.2,f,Other Cities,34.10933,-118.0731,Private room,1.0,1,49,1,364,1,3.00,f,2.000000
2,within an hour,1,t,Other Cities,34.01559,-118.4941,Entire home/apt,1.0,0,231,5,193,44,4.93,t,2.000000
3,within a few hours,1,f,City of Los Angeles,33.99017,-118.4225,Entire home/apt,1.0,2,110,30,117,368,4.61,f,1.400000
4,within an hour,1,t,City of Los Angeles,33.98258,-118.4643,Entire home/apt,3.5,4,1179,2,356,35,4.97,f,1.500000
5,within an hour,1,t,City of Los Angeles,34.08817,-118.3655,Entire home/apt,4.5,4,1244,3,141,10,5.00,f,2.000000
6,within an hour,1,t,Other Cities,34.09130,-118.3519,Entire home/apt,1.0,2,213,2,244,23,4.74,t,1.666667


In [54]:

data %>% count(host_response_time) %>% mutate(prop = n / nrow(data))

host_response_time,n,prop
<chr>,<int>,<dbl>
a few days or more,457,0.01767960
within a day,1181,0.04568842
within a few hours,3362,0.13006306
within an hour,20849,0.80656892


In [52]:
data <- data %>%
    filter(host_response_time != 'N/A')
data %>% count(host_response_time) %>% mutate(prop = n / nrow(data))

host_response_time,n,prop
<chr>,<int>,<dbl>
a few days or more,457,0.01767960
within a day,1181,0.04568842
within a few hours,3362,0.13006306
within an hour,20849,0.80656892


In [32]:
colnames(airpnp)

[1] "id"                           "name"                        
 [3] "host_id"                      "host_name"                   
 [5] "host_since"                   "host_response_time"          
 [7] "host_response_rate"           "host_is_superhost"           
 [9] "neighbourhood_cleansed"       "neighbourhood_group_cleansed"
[11] "latitude"                     "longitude"                   
[13] "property_type"                "room_type"                   
[15] "accommodates"                 "bathrooms"                   
[17] "bedrooms"                     "beds"                        
[19] "price"                        "minimum_nights"              
[21] "availability_365"             "number_of_reviews"           
[23] "review_scores_rating"         "license"                     
[25] "instant_bookable"

In [33]:
airpnp <- airpnp %>%
    select(-id, -name, -host_id, -host_name, -host_since, -neighbourhood_cleansed, -property_type, -license)

In [35]:
colSums(is.na(airpnp))

host_response_time           host_response_rate 
                           0                            0 
           host_is_superhost neighbourhood_group_cleansed 
                           0                            0 
                    latitude                    longitude 
                           0                            0 
                   room_type                 accommodates 
                           0                            0 
                   bathrooms                     bedrooms 
                        8239                         3039 
                        beds                        price 
                        8334                         8237 
              minimum_nights             availability_365 
                           0                            0 
           number_of_reviews         review_scores_rating 
                           0                        12146 
            instant_bookable 
                           0

In [36]:
airpnp <- na.omit(airpnp)
dim(airpnp)

[1] 28045    17

In [37]:
str(airpnp)

'data.frame':	28045 obs. of  17 variables:
 $ host_response_time          : chr  "a few days or more" "within an hour" "within a few hours" "within an hour" ...
 $ host_response_rate          : chr  "0.2" "1" "1" "1" ...
 $ host_is_superhost           : chr  "f" "t" "f" "t" ...
 $ neighbourhood_group_cleansed: chr  "Other Cities" "Other Cities" "City of Los Angeles" "City of Los Angeles" ...
 $ latitude                    : num  34.1 34 34 34 34.1 ...
 $ longitude                   : num  -118 -118 -118 -118 -118 ...
 $ room_type                   : chr  "Private room" "Entire home/apt" "Entire home/apt" "Entire home/apt" ...
 $ accommodates                : int  2 2 7 6 8 5 4 3 2 6 ...
 $ bathrooms                   : num  1 1 1 3.5 4.5 1 1 1 1 2 ...
 $ bedrooms                    : int  1 0 2 4 4 2 2 1 1 2 ...
 $ beds                        : int  1 1 5 4 4 3 3 1 1 3 ...
 $ price                       : int  49 231 110 1179 1244 213 95 298 110 282 ...
 $ minimum_nights              :

In [38]:
airpnp["beds_per_accomodate"] = airpnp$accommodates / airpnp$beds

In [39]:
colnames(airpnp)

[1] "host_response_time"           "host_response_rate"          
 [3] "host_is_superhost"            "neighbourhood_group_cleansed"
 [5] "latitude"                     "longitude"                   
 [7] "room_type"                    "accommodates"                
 [9] "bathrooms"                    "bedrooms"                    
[11] "beds"                         "price"                       
[13] "minimum_nights"               "availability_365"            
[15] "number_of_reviews"            "review_scores_rating"        
[17] "instant_bookable"             "beds_per_accomodate"

In [40]:
airpnp <- airpnp %>%
    select(-beds, -accommodates)

In [41]:
colnames(airpnp)

[1] "host_response_time"           "host_response_rate"          
 [3] "host_is_superhost"            "neighbourhood_group_cleansed"
 [5] "latitude"                     "longitude"                   
 [7] "room_type"                    "bathrooms"                   
 [9] "bedrooms"                     "price"                       
[11] "minimum_nights"               "availability_365"            
[13] "number_of_reviews"            "review_scores_rating"        
[15] "instant_bookable"             "beds_per_accomodate"